# Calculates LayerCAM for the Multi-label Dataset
- on two different baseline networks

In [ ]:
from libraries_multilabel.energyPointGame import energy_point_game_mask
from libraries_multilabel.bcosconv2d import NormedConv2d
from torchvision import models, transforms
from torchvision.models import ResNet50_Weights, resnet50

import random
import numpy as np
import torch
import pandas as pd
import os
from PIL import Image
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from libraries_multilabel.bcoslinear import BcosLinear
from libraries_multilabel.MultiLabelExplanationWrapper import MultiLabelModelWrapper
from libraries_multilabel.MultiLabelDatasets import MultiLabelDatasetID
from cam.layercam import LayerCAM
import torch.nn as nn

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path configurations
image_folder = r"D:\vinbigdata-chest-xray-abnormalities-detection\train_png_224"
csv_path_boxes = r"D:\vinbigdata-chest-xray-abnormalities-detection\train224.csv"
csv_path = r"G:\Meine Ablage\Universität\Master Thesis\Multi-Classification\training\multilabel_dataset.csv"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Multi-Classification\training\vinbigdata_5fold_splits.pkl"

# Initialize datasets and transforms
data = pd.read_csv(csv_path)
data_boxes = pd.read_csv(csv_path_boxes)
with open(splits_path, 'rb') as f:
    splits = pickle.load(f)
    
transform = transforms.Compose([transforms.ToTensor()])
class_names = ["Aortic enlargement", "Atelectasis", "Calcification", "Cardiomegaly",
    "Consolidation", "ILD", "Infiltration", "Lung Opacity",
    "Nodule/Mass", "Other lesion", "Pleural effusion", "Pleural thickening",
    "Pneumothorax", "Pulmonary fibrosis"]

class_names_extended = ["Aortic enlargement", "Atelectasis", "Calcification", "Cardiomegaly",
    "Consolidation", "ILD", "Infiltration", "Lung Opacity",
    "Nodule/Mass", "Other lesion", "Pleural effusion", "Pleural thickening",
    "Pneumothorax", "Pulmonary fibrosis", "Average"]

seeds = [0, 1]
seed_results = {
    name: {
        'proportions': [],
        'correct': [],
        'incorrect': [],
    } for name in class_names
}

seed_results_avg = { 
        'proportions': [],
        'correct': [],
        'incorrect': []
        }

for seed in seeds:
    print(f"\n=== Seed {seed+1} Results ===")
    all_fold_results = []
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    fold_average_results = {name: {
            'avg_proportion': [],
            'avg_correct': [],
            'avg_incorrect': [],
            'correct_count': [],
            'incorrect_count': []
        } for name in class_names}

    fold_avg_row_results = {'proportions': [],
        'correct': [],
        'incorrect': []
        }

    # Process all 5 folds
    for fold in range(5):
        print(f"Processing fold {fold+1} of 5")
        model_path = fr"C:\Users\Admin\Documents\MasterThesis\results\VinBigData\ResNet_Baseline\no_nosamp\seed_{seed}\pneumonia_detection_model_resnet_baseline_bestf1_{fold+1}.pth"
        
        # Initialize results storage        
        fold_all_outputs = {name: {
            'proportions': [],
            'correct': [],
            'incorrect': [],
            'correct_count': 0,
            'incorrect_count': 0
        } for name in class_names} # need to be averaged for final_fold_result


        model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        model.fc = nn.Linear(model.fc.in_features, 14)
        model.load_state_dict(torch.load(model_path))
        model.to(device)
        model_dict = dict(
            type="resnet50",
            layer_name="layer4",
            arch=model,
            target_layer=model.layer4[2].conv3 # Example: last layer of ResNet's layer4  ###### IN BCOS:     target_layer=model.layer4[-1].conv3  # Example: last layer of ResNet's layer4  ### double check!

        )
        cam = LayerCAM(model_dict)
        
        val_idx = splits[fold][1]
        val_data = data.iloc[val_idx]
        val_dataset = MultiLabelDatasetID(val_data, image_folder, transform=transform)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

        multiLabelWrapper = MultiLabelModelWrapper(model)
        model.eval()
        multiLabelWrapper.model.eval()

        with torch.enable_grad():
            for images, labels, image_ids in val_loader:
                images, labels = images.to(device), labels.to(device)                
                mask = torch.zeros((224, 224), dtype=torch.int32)
                for image, label, image_id in zip(images, labels, image_ids):
                    image = image[None]  # Add batch dimension
                    output = model(image)
                    probs = torch.sigmoid(output)
                    
                    binary_preds = (probs > 0.5).int()                    
                    
                    for class_idx, class_name in enumerate(class_names):
                        filtered_rows = data_boxes[(data_boxes['image_id'] == image_id)]
                        labels_row = data[data['image_id'] == image_id]
                        
                        if not filtered_rows.empty and not labels_row.empty and labels_row[class_name].iloc[0] == 1:
                            prediction = binary_preds[0][class_idx]
                            contribution_map = cam(image, class_idx=class_idx)
                            contribution_map[contribution_map < 0] = 0
                            contribution_map = contribution_map.squeeze(0).squeeze(0)
                            
                            mask.zero_()
                            for _, row in filtered_rows.iterrows():
                                x_min = int(row["x_min"])
                                y_min = int(row["y_min"])
                                x_max = int(row["x_max"])
                                y_max = int(row["y_max"])
                                mask[y_min:y_max, x_min:x_max] = 1.0
                            
                            # Calculate energy point game metric
                            ebpg_result = energy_point_game_mask(mask.cpu(), contribution_map.cpu())
                            
                            # Update results
                            fold_all_outputs[class_name]['proportions'].append(ebpg_result)
                            if prediction == 1:
                                fold_all_outputs[class_name]['correct'].append(ebpg_result)
                                fold_all_outputs[class_name]['correct_count'] += 1
                            else:
                                fold_all_outputs[class_name]['incorrect'].append(ebpg_result)
                                fold_all_outputs[class_name]['incorrect_count'] += 1
                                
                        
        for class_name in class_names:
            class_fold_result = fold_all_outputs[class_name]
            
            avg_total = np.mean(class_fold_result['proportions']) if class_fold_result['proportions'] else 0
            avg_correct = np.mean(class_fold_result['correct']) if class_fold_result['correct'] else 0
            avg_incorrect = np.mean(class_fold_result['incorrect']) if class_fold_result['incorrect'] else 0
            
            fold_average_results[class_name]['avg_proportion'].append(round(avg_total, 4))
            fold_average_results[class_name]['avg_correct'].append(round(avg_correct, 4))
            fold_average_results[class_name]['avg_incorrect'].append(round(avg_incorrect, 4))
            fold_average_results[class_name]['correct_count'].append(class_fold_result['correct_count'])
            fold_average_results[class_name]['incorrect_count'].append(class_fold_result['incorrect_count'])

        rows = []
        for class_name in class_names:
            rows.append({
                'Class': class_name,
                'avg_proportion': fold_average_results[class_name]['avg_proportion'][fold],
                'avg_correct': fold_average_results[class_name]['avg_correct'][fold],
                'avg_incorrect': fold_average_results[class_name]['avg_incorrect'][fold],
                'correct_count': fold_average_results[class_name]['correct_count'][fold],
                'incorrect_count': fold_average_results[class_name]['incorrect_count'][fold]
            })

        df = pd.DataFrame(rows)

        # add an average row
        avg_row = {
            'Class': 'Average',
            'avg_proportion': round(df['avg_proportion'].mean(), 4),
            'avg_correct': round(df['avg_correct'].mean(), 4),
            'avg_incorrect': round(df['avg_incorrect'].mean(), 4),
            'correct_count': int(df['correct_count'].sum()),        
            'incorrect_count': int(df['incorrect_count'].sum()),    
        }
        fold_avg_row_results['proportions'].append(avg_row['avg_proportion'])
        fold_avg_row_results['correct'].append(avg_row['avg_correct'])
        fold_avg_row_results['incorrect'].append(avg_row['avg_incorrect'])

        
        df = pd.concat([
            df,
            pd.DataFrame([avg_row])  # Wrap in list to create 1-row DataFrame
        ], ignore_index=True)

        print(f"\nEnergy-Based Pointing Game Results for Fold {fold+1}:")
        print(df.to_string(index=False))        
        
        if fold == 4:
            for class_name in class_names:
                seed_results[class_name]['proportions'].append(np.mean(fold_average_results[class_name]['avg_proportion']))
                seed_results[class_name]['correct'].append(np.mean(fold_average_results[class_name]['avg_correct']))
                seed_results[class_name]['incorrect'].append(np.mean(fold_average_results[class_name]['avg_incorrect']))   
                
            seed_results_avg['proportions'].append(np.mean(fold_avg_row_results['proportions']))
            seed_results_avg['correct'].append(np.mean(fold_avg_row_results['correct']))
            seed_results_avg['incorrect'].append(np.mean(fold_avg_row_results['incorrect']))

                
        
                    

def calculate_stats(values):
    return f"{np.mean(values):.4f} ± {np.std(values):.4f}"

all_props = []
all_corrs = []
all_incorrects = []

print("\n\n=== Final Cross-Seed Averages ===")
for class_name in class_names:
    print(f"\nClass: {class_name}")
    print("Proportion:", calculate_stats(seed_results[class_name]['proportions']))
    print("Correct   :", calculate_stats(seed_results[class_name]['correct']))
    print("Incorrect :", calculate_stats(seed_results[class_name]['incorrect']))

print("\n=== Final Cross-Seed Average (across all classes) ===")
print("Proportion:", calculate_stats(seed_results_avg['proportions']))
print("Correct   :", calculate_stats(seed_results_avg['correct']))
print("Incorrect :", calculate_stats(seed_results_avg['incorrect']))



=== Seed 1 Results ===
Processing fold 1 of 5


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ..\aten\src\ATen\cuda\CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



Energy-Based Pointing Game Results for Fold 1:
             Class  avg_proportion  avg_correct  avg_incorrect  correct_count  incorrect_count
Aortic enlargement          0.1502       0.1538         0.1260            533               78
       Atelectasis          0.3101       0.2141         0.3154              2               36
     Calcification          0.1694       0.2528         0.1568             12               79
      Cardiomegaly          0.1860       0.1920         0.1564            382               78
     Consolidation          0.2867       0.3668         0.2457             24               47
               ILD          0.3023       0.3901         0.2548             27               50
      Infiltration          0.2376       0.2771         0.2066             54               69
      Lung Opacity          0.2097       0.2945         0.1350            124              141
       Nodule/Mass          0.1309       0.1413         0.1283             33              132
  

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



Energy-Based Pointing Game Results for Fold 2:
             Class  avg_proportion  avg_correct  avg_incorrect  correct_count  incorrect_count
Aortic enlargement          0.1569       0.1544         0.1705            518               94
       Atelectasis          0.2539       0.4085         0.2451              2               35
     Calcification          0.1671       0.2748         0.1566              8               82
      Cardiomegaly          0.1983       0.1998         0.1888            399               61
     Consolidation          0.2981       0.3928         0.2497             24               47
               ILD          0.3373       0.4220         0.3154             16               62
      Infiltration          0.2205       0.3178         0.1748             39               83
      Lung Opacity          0.2198       0.2951         0.1191            151              113
       Nodule/Mass          0.1507       0.1484         0.1513             31              134
  

...


Energy-Based Pointing Game Results for Fold 3:
             Class  avg_proportion  avg_correct  avg_incorrect  correct_count  incorrect_count
Aortic enlargement          0.1567       0.1591         0.1428            530               91
       Atelectasis          0.2521       0.3318         0.2367              6               31
     Calcification          0.1454       0.2454         0.1270             14               76
      Cardiomegaly          0.1900       0.1895         0.1919            371               89
     Consolidation          0.2641       0.3733         0.2291             17               53
               ILD          0.3413       0.3993         0.3001             32               45
      Infiltration          0.2273       0.3222         0.1925             33               90
      Lung Opacity          0.2055       0.2916         0.1338            120              144
       Nodule/Mass          0.1470       0.1517         0.1461             26              139
  

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



Energy-Based Pointing Game Results for Fold 4:
             Class  avg_proportion  avg_correct  avg_incorrect  correct_count  incorrect_count
Aortic enlargement          0.1561       0.1590         0.1462            471              141
       Atelectasis          0.2409       0.0000         0.2409              0               37
     Calcification          0.1615       0.2331         0.1590              3               87
      Cardiomegaly          0.2458       0.2451         0.2535            421               39
     Consolidation          0.2772       0.3304         0.2374             30               40
               ILD          0.3365       0.5692         0.3018             10               67
      Infiltration          0.2485       0.3240         0.2204             33               89
      Lung Opacity          0.2082       0.2451         0.1357            175               89
       Nodule/Mass          0.1607       0.1548         0.1620             30              135
  

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



Energy-Based Pointing Game Results for Fold 5:
             Class  avg_proportion  avg_correct  avg_incorrect  correct_count  incorrect_count
Aortic enlargement          0.1468       0.1463         0.1491            491              120
       Atelectasis          0.2948       0.2991         0.2944              3               34
     Calcification          0.1601       0.1838         0.1590              4               87
      Cardiomegaly          0.1961       0.1999         0.1806            371               89
     Consolidation          0.2822       0.4021         0.2669              8               63
               ILD          0.3399       0.4653         0.2898             22               55
      Infiltration          0.2296       0.3189         0.1955             34               89
      Lung Opacity          0.2108       0.3023         0.1478            108              157
       Nodule/Mass          0.1440       0.1430         0.1443             36              130
  

In [ ]:
from libraries_multilabel.energyPointGame import energy_point_game_mask
from libraries_multilabel.bcosconv2d import NormedConv2d
from torchvision import models, transforms
from torchvision.models import ResNet50_Weights, resnet50



import random
import numpy as np
import torch
import pandas as pd
import os
from PIL import Image
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from libraries_multilabel.bcoslinear import BcosLinear
from libraries_multilabel.MultiLabelExplanationWrapper import MultiLabelModelWrapper
from libraries_multilabel.MultiLabelDatasets import MultiLabelDatasetID
from cam.layercam import LayerCAM
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_folder = r"D:\vinbigdata-chest-xray-abnormalities-detection\train_png_224"
csv_path_boxes = r"D:\vinbigdata-chest-xray-abnormalities-detection\train224.csv"
csv_path = r"G:\Meine Ablage\Universität\Master Thesis\Multi-Classification\training\multilabel_dataset.csv"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Multi-Classification\training\vinbigdata_5fold_splits.pkl"

# Initialize datasets and transforms
data = pd.read_csv(csv_path)
data_boxes = pd.read_csv(csv_path_boxes)
with open(splits_path, 'rb') as f:
    splits = pickle.load(f)
    
transform = transforms.Compose([transforms.ToTensor()])
class_names = ["Aortic enlargement", "Atelectasis", "Calcification", "Cardiomegaly",
    "Consolidation", "ILD", "Infiltration", "Lung Opacity",
    "Nodule/Mass", "Other lesion", "Pleural effusion", "Pleural thickening",
    "Pneumothorax", "Pulmonary fibrosis"]

class_names_extended = ["Aortic enlargement", "Atelectasis", "Calcification", "Cardiomegaly",
    "Consolidation", "ILD", "Infiltration", "Lung Opacity",
    "Nodule/Mass", "Other lesion", "Pleural effusion", "Pleural thickening",
    "Pneumothorax", "Pulmonary fibrosis", "Average"]

seeds = [0, 1]
seed_results = {
    name: {
        'proportions': [],
        'correct': [],
        'incorrect': [],
    } for name in class_names
}

seed_results_avg = { 
        'proportions': [],
        'correct': [],
        'incorrect': []
        }

for seed in seeds:
    print(f"\n=== Seed {seed+1} Results ===")
    all_fold_results = []
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    fold_average_results = {name: {
            'avg_proportion': [],
            'avg_correct': [],
            'avg_incorrect': [],
            'correct_count': [],
            'incorrect_count': []
        } for name in class_names}

    fold_avg_row_results = {'proportions': [],
        'correct': [],
        'incorrect': []
        }

    # Process all 5 folds
    for fold in range(5):
        print(f"Processing fold {fold+1} of 5")
        model_path = fr"C:\Users\Admin\Documents\MasterThesis\results\VinBigData\ResNet_Baseline\light_oversamp\seed_{seed}\pneumonia_detection_model_resnet_baseline_bestf1_{fold+1}.pth"
        
        # Initialize results storage        
        fold_all_outputs = {name: {
            'proportions': [],
            'correct': [],
            'incorrect': [],
            'correct_count': 0,
            'incorrect_count': 0
        } for name in class_names} # need to be averaged for final_fold_result


        model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        model.fc = nn.Linear(model.fc.in_features, 14)
        model.load_state_dict(torch.load(model_path))
        model.to(device)
        model_dict = dict(
            type="resnet50",
            layer_name="layer4",
            arch=model,
            target_layer=model.layer4[2].conv3 # Example: last layer of ResNet's layer4  ###### IN BCOS:     target_layer=model.layer4[-1].conv3  # Example: last layer of ResNet's layer4  ### double check!

        )
        cam = LayerCAM(model_dict)
        
        val_idx = splits[fold][1]
        val_data = data.iloc[val_idx]
        val_dataset = MultiLabelDatasetID(val_data, image_folder, transform=transform)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

        multiLabelWrapper = MultiLabelModelWrapper(model)
        model.eval()
        multiLabelWrapper.model.eval()

        with torch.enable_grad():
            for images, labels, image_ids in val_loader:
                images, labels = images.to(device), labels.to(device)                
                mask = torch.zeros((224, 224), dtype=torch.int32)
                for image, label, image_id in zip(images, labels, image_ids):
                    image = image[None]  # Add batch dimension
                    output = model(image)
                    probs = torch.sigmoid(output)
                    
                    binary_preds = (probs > 0.5).int()                    
                    
                    for class_idx, class_name in enumerate(class_names):
                        filtered_rows = data_boxes[(data_boxes['image_id'] == image_id)]
                        labels_row = data[data['image_id'] == image_id]
                        
                        if not filtered_rows.empty and not labels_row.empty and labels_row[class_name].iloc[0] == 1:
                            prediction = binary_preds[0][class_idx]
                            contribution_map = cam(image, class_idx=class_idx)
                            contribution_map[contribution_map < 0] = 0
                            contribution_map = contribution_map.squeeze(0).squeeze(0)
                            
                            mask.zero_()
                            for _, row in filtered_rows.iterrows():
                                x_min = int(row["x_min"])
                                y_min = int(row["y_min"])
                                x_max = int(row["x_max"])
                                y_max = int(row["y_max"])
                                mask[y_min:y_max, x_min:x_max] = 1.0
                            
                            # Calculate energy point game metric
                            ebpg_result = energy_point_game_mask(mask.cpu(), contribution_map.cpu())
                            
                            # Update results
                            fold_all_outputs[class_name]['proportions'].append(ebpg_result)
                            if prediction == 1:
                                fold_all_outputs[class_name]['correct'].append(ebpg_result)
                                fold_all_outputs[class_name]['correct_count'] += 1
                            else:
                                fold_all_outputs[class_name]['incorrect'].append(ebpg_result)
                                fold_all_outputs[class_name]['incorrect_count'] += 1
                                
                        
        for class_name in class_names:
            class_fold_result = fold_all_outputs[class_name]
            
            avg_total = np.mean(class_fold_result['proportions']) if class_fold_result['proportions'] else 0
            avg_correct = np.mean(class_fold_result['correct']) if class_fold_result['correct'] else 0
            avg_incorrect = np.mean(class_fold_result['incorrect']) if class_fold_result['incorrect'] else 0
            
            fold_average_results[class_name]['avg_proportion'].append(round(avg_total, 4))
            fold_average_results[class_name]['avg_correct'].append(round(avg_correct, 4))
            fold_average_results[class_name]['avg_incorrect'].append(round(avg_incorrect, 4))
            fold_average_results[class_name]['correct_count'].append(class_fold_result['correct_count'])
            fold_average_results[class_name]['incorrect_count'].append(class_fold_result['incorrect_count'])

        rows = []
        for class_name in class_names:
            rows.append({
                'Class': class_name,
                'avg_proportion': fold_average_results[class_name]['avg_proportion'][fold],
                'avg_correct': fold_average_results[class_name]['avg_correct'][fold],
                'avg_incorrect': fold_average_results[class_name]['avg_incorrect'][fold],
                'correct_count': fold_average_results[class_name]['correct_count'][fold],
                'incorrect_count': fold_average_results[class_name]['incorrect_count'][fold]
            })

        df = pd.DataFrame(rows)

        # add an average row
        avg_row = {
            'Class': 'Average',
            'avg_proportion': round(df['avg_proportion'].mean(), 4),
            'avg_correct': round(df['avg_correct'].mean(), 4),
            'avg_incorrect': round(df['avg_incorrect'].mean(), 4),
            'correct_count': int(df['correct_count'].sum()),        
            'incorrect_count': int(df['incorrect_count'].sum()),    
        }
        fold_avg_row_results['proportions'].append(avg_row['avg_proportion'])
        fold_avg_row_results['correct'].append(avg_row['avg_correct'])
        fold_avg_row_results['incorrect'].append(avg_row['avg_incorrect'])

        
        df = pd.concat([
            df,
            pd.DataFrame([avg_row])  # Wrap in list to create 1-row DataFrame
        ], ignore_index=True)

        print(f"\nEnergy-Based Pointing Game Results for Fold {fold+1}:")
        print(df.to_string(index=False))        
        
        if fold == 4:
            for class_name in class_names:
                seed_results[class_name]['proportions'].append(np.mean(fold_average_results[class_name]['avg_proportion']))
                seed_results[class_name]['correct'].append(np.mean(fold_average_results[class_name]['avg_correct']))
                seed_results[class_name]['incorrect'].append(np.mean(fold_average_results[class_name]['avg_incorrect']))   
                
            seed_results_avg['proportions'].append(np.mean(fold_avg_row_results['proportions']))
            seed_results_avg['correct'].append(np.mean(fold_avg_row_results['correct']))
            seed_results_avg['incorrect'].append(np.mean(fold_avg_row_results['incorrect']))

                
        
                    

def calculate_stats(values):
    return f"{np.mean(values):.4f} ± {np.std(values):.4f}"

all_props = []
all_corrs = []
all_incorrects = []

print("\n\n=== Final Cross-Seed Averages ===")
for class_name in class_names:
    print(f"\nClass: {class_name}")
    print("Proportion:", calculate_stats(seed_results[class_name]['proportions']))
    print("Correct   :", calculate_stats(seed_results[class_name]['correct']))
    print("Incorrect :", calculate_stats(seed_results[class_name]['incorrect']))

print("\n=== Final Cross-Seed Average (across all classes) ===")
print("Proportion:", calculate_stats(seed_results_avg['proportions']))
print("Correct   :", calculate_stats(seed_results_avg['correct']))
print("Incorrect :", calculate_stats(seed_results_avg['incorrect']))



=== Seed 1 Results ===
Processing fold 1 of 5


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



Energy-Based Pointing Game Results for Fold 1:
             Class  avg_proportion  avg_correct  avg_incorrect  correct_count  incorrect_count
Aortic enlargement          0.1645       0.1624         0.1900            563               48
       Atelectasis          0.2819       0.2846         0.2802             15               23
     Calcification          0.1709       0.2323         0.1525             21               70
      Cardiomegaly          0.2021       0.1977         0.2347            406               54
     Consolidation          0.2535       0.3344         0.2148             23               48
               ILD          0.2885       0.3234         0.2696             27               50
      Infiltration          0.2216       0.2467         0.2007             56               67
      Lung Opacity          0.2070       0.2689         0.1565            119              146
       Nodule/Mass          0.1374       0.1131         0.1454             41              124
  

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



Energy-Based Pointing Game Results for Fold 2:
             Class  avg_proportion  avg_correct  avg_incorrect  correct_count  incorrect_count
Aortic enlargement          0.1764       0.1676         0.2409            538               74
       Atelectasis          0.2518       0.2648         0.2408             17               20
     Calcification          0.1772       0.2256         0.1643             19               71
      Cardiomegaly          0.2057       0.2010         0.2419            407               53
     Consolidation          0.2770       0.3131         0.2489             31               40
               ILD          0.3464       0.4224         0.3146             23               55
      Infiltration          0.2147       0.2753         0.1754             48               74
      Lung Opacity          0.2187       0.3052         0.1455            121              143
       Nodule/Mass          0.1641       0.1553         0.1676             47              118
  

...


Energy-Based Pointing Game Results for Fold 3:
             Class  avg_proportion  avg_correct  avg_incorrect  correct_count  incorrect_count
Aortic enlargement          0.1702       0.1673         0.1923            549               72
       Atelectasis          0.2625       0.2759         0.2576             10               27
     Calcification          0.1436       0.1586         0.1398             18               72
      Cardiomegaly          0.1984       0.1975         0.2047            408               52
     Consolidation          0.2471       0.3310         0.1945             27               43
               ILD          0.3295       0.3986         0.2719             35               42
      Infiltration          0.2103       0.2689         0.1701             50               73
      Lung Opacity          0.2051       0.2546         0.1620            123              141
       Nodule/Mass          0.1566       0.1637         0.1545             38              127
  

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



Energy-Based Pointing Game Results for Fold 4:
             Class  avg_proportion  avg_correct  avg_incorrect  correct_count  incorrect_count
Aortic enlargement          0.1654       0.1638         0.1816            555               57
       Atelectasis          0.2332       0.2259         0.2352              8               29
     Calcification          0.1593       0.1507         0.1618             20               70
      Cardiomegaly          0.2074       0.2066         0.2134            408               52
     Consolidation          0.2551       0.3177         0.2264             22               48
               ILD          0.3103       0.4334         0.2642             21               56
      Infiltration          0.2271       0.2634         0.1993             53               69
      Lung Opacity          0.1991       0.2438         0.1636            117              147
       Nodule/Mass          0.1612       0.1869         0.1487             54              111
  

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



Energy-Based Pointing Game Results for Fold 5:
             Class  avg_proportion  avg_correct  avg_incorrect  correct_count  incorrect_count
Aortic enlargement          0.1853       0.1885         0.1399            571               40
       Atelectasis          0.2974       0.3077         0.2865             19               18
     Calcification          0.1683       0.1906         0.1578             29               62
      Cardiomegaly          0.2274       0.2266         0.2309            376               84
     Consolidation          0.2762       0.2897         0.2589             40               31
               ILD          0.3437       0.4980         0.3232              9               68
      Infiltration          0.2328       0.2817         0.1761             66               57
      Lung Opacity          0.2181       0.2473         0.1606            176               89
       Nodule/Mass          0.1617       0.1448         0.1760             76               90
  